## 1.0 Setting up the environment

### 1.1 Importing libraries

We are only using the following libraries for this project:
- **pandas** for data manipulation
- **requests** for downloading the data
- **zipfile** for unzipping the data
- **io** for reading the data
- **os** for checking the existence of files

In [120]:
import pandas as pd
import requests
import zipfile
import io
import os


### 1.2 Check for gtfs.zip and extract

We are using the gtfs.zip file from the [LACMTA GTFS Data Repository]()
- If the file is not present, download it
- If the file is present, extract it

In [121]:

def download_gtfs_and_extract_zip(url, zip_file_name,output_folder):
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(output_folder)
    print(f'Extracted {zip_file_name} to {output_folder}')

def check_if_gtfs_files_exist(target_folder):
    gtfs_files = ['agency.txt', 'calendar.txt', 'calendar_dates.txt', 'routes.txt', 'shapes.txt', 'stop_times.txt', 'stops.txt', 'trips.txt']
    for file in gtfs_files:
        if not os.path.isfile(f'{target_folder}/{file}'):
            print(f'{file} is missing from {target_folder}')
            return False
    return True

gtfs_url = 'https://github.com/LACMTA/los-angeles-regional-gtfs/raw/main/lacmta/current-base/gtfs_bus.zip'

if check_if_gtfs_files_exist('input'):
    print('GTFS files already exist')
else:
    download_gtfs_and_extract_zip(gtfs_url, 'gtfs_bus.zip', 'input')


GTFS files already exist


### 1.3 Loading the data

We are using the following GTFS Static datasets:
- **trips.txt** for the trips information: 
  - route_id, service_id, trip_id, trip_headsign, direction_id, shape_id
- **stop_times.txt** for the stop times information:
  - trip_id, stop_id, stop_sequence

In [122]:
bus_gtfs_trips = pd.read_csv('input/trips.txt')
bus_gtfs_stop_times = pd.read_csv('input/stop_times.txt')

C:\Users\hikou\AppData\Local\Temp\ipykernel_21252\1517197612.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  bus_gtfs_stop_times = pd.read_csv('input/stop_times.txt')


## 2.0 Joining and grouping the data

We will perform following tasks:
1. Join the `trips` and `stop_times` data to get each `stop_id` and `stop_sequence` for each trip
2. Group the data by `route_id` and `direction_id` to get the number of `shape_ids` for each route and stop
3. Count the number of `shapes_id`s for each `route_id` and `direction_id`
4. Count the number of `shape_id`s for each `route_id` and `stop_id` combination
5. Filter the data to only include stops with more than 1 `shape_id`


### 2.1 Joining the trips and stop times data

We begin by joining the two datasets on the `trip_id` column. To get a list of all the `stop_id`s for each `route_id`. 

This will give us a dataframe called `simplified_trips_join_stop_times` with the following columns:
- `route_id`
- `service_id`
- `trip_id`
- `direction_id`
- `shape_id`
- `stop_id`
- `stop_sequence`

In [93]:
trips_joined_stop_times = pd.merge(bus_gtfs_trips, bus_gtfs_stop_times, on='trip_id', how='inner')
group_join_by_stop_data = trips_joined_stop_times.groupby(['route_id','direction_id','shape_id','stop_id','stop_sequence']).size().reset_index(name='count')
order_by_shape_id_stop_sequence = group_join_by_stop_data.sort_values(by=['shape_id','stop_sequence'], ascending=True)
simplified_trips_join_stop_times = order_by_shape_id_stop_sequence[['route_id','direction_id','shape_id','stop_id', 'stop_sequence']]

,route_id,direction_id,shape_id,stop_id,stop_sequence
11,10-13168,0,100750_JUNE23,3202,1
14,10-13168,0,100750_JUNE23,3217,2
20,10-13168,0,100750_JUNE23,3232,3
16,10-13168,0,100750_JUNE23,3227,4
19,10-13168,0,100750_JUNE23,3231,5
...,...,...,...,...,...
41110,DSE-US,1,DSE-US-US-CF,142043,2
41109,DSE-US,1,DSE-US-US-CF,2155,3
41114,DSE-US,1,DSE-US-US-TD,63500004,1
41113,DSE-US,1,DSE-US-US-TD,63500002,2


### 2.2 Grouping the `stop_id`s by `route_id` and `direction_id`

We then group the `stop_id`s by `route_id` and `direction_id` to unique list of `shape_id`s for each `route_id` and `direction_id` combination.


In [95]:
group_route_id_by_distinct_direction_id_shape_ids = simplified_trips_join_stop_times.groupby(['route_id','direction_id'])['shape_id'].unique().reset_index(name='shape_ids')
group_by_direction_id_and_shape_ids = simplified_trips_join_stop_times.groupby(['route_id','direction_id']).agg({'shape_id': pd.Series.nunique}).reset_index()

,route_id,direction_id,number_of_shape_ids
0,10-13168,0,14
1,10-13168,1,9
2,102-13168,0,4
3,102-13168,1,3
4,105-13168,0,4
...,...,...,...
227,96-13168,1,1
228,DSE-HG,0,1
229,DSE-HG,1,1
230,DSE-US,0,1


### 2.3 Count up the number of shapes for each `route_id` and `direction_id`

As an intermediate step, we count up the number of shapes for each `route_id` and `direction_id` combination. This will give us a dataframe called `group_by_direction_id_and_shape_ids` with the following columns:
- `route_id`
- `direction_id`
- `number_of_shape_ids`


In [102]:
group_by_direction_id_and_shape_ids.columns = ['route_id','direction_id','number_of_shape_ids']
group_by_direction_id_and_shape_ids

,route_id,direction_id,number_of_shape_ids
0,10-13168,0,14
1,10-13168,1,9
2,102-13168,0,4
3,102-13168,1,3
4,105-13168,0,4
...,...,...,...
227,96-13168,1,1
228,DSE-HG,0,1
229,DSE-HG,1,1
230,DSE-US,0,1


### 2.4 Group by `route_id` and `direction_id`

We then count the number of `shape_id`s for each `route_id` and `stop_id` combination.

This will give us a column called `number_of_shape_ids` which is the number of shapes for each `route_id` and `stop_id` combination.

In [98]:
group_by_route_id_and_direction_id = group_by_direction_id_and_shape_ids.groupby(['route_id','direction_id'])['number_of_shape_ids'].unique().reset_index(name='shape_ids')
group_by_route_id_and_direction_id.rename(columns={'shape_ids': 'number_of_shapes_per_direction'}, inplace=True)
group_by_route_id_and_direction_id['number_of_shapes_per_direction'] = group_by_route_id_and_direction_id['number_of_shapes_per_direction'].apply(lambda x: x[0])
group_by_route_id_and_direction_id


,route_id,direction_id,number_of_shapes_per_direction
0,10-13168,0,14
1,10-13168,1,9
2,102-13168,0,4
3,102-13168,1,3
4,105-13168,0,4
...,...,...,...
227,96-13168,1,1
228,DSE-HG,0,1
229,DSE-HG,1,1
230,DSE-US,0,1


### 2.5 Filter out the `route_id`s with only one shape

We then filter out the `route_id`s with only one shape into a dataframe called `route_ids_number_of_shapes_per_direction_greater_than_1`

In [103]:
route_ids_number_of_shapes_per_direction_greater_than_1 = group_by_direction_id_and_shape_ids[group_by_route_id_and_direction_id['number_of_shapes_per_direction'] > 1]
route_ids_number_of_shapes_per_direction_greater_than_1

,route_id,direction_id,number_of_shape_ids
0,10-13168,0,14
1,10-13168,1,9
2,102-13168,0,4
3,102-13168,1,3
4,105-13168,0,4
...,...,...,...
221,910-13168,1,2
222,92-13168,0,6
223,92-13168,1,7
224,94-13168,0,2


### 3.0 Merging the joined data with the number of shapes data


### 3.1 Get a list of all the `stop_id`s for each `route_id`

We need to get a list of all the `stop_id`s for each `route_id` to get the differences between each `route_id` and `stop_id` combination.

In [104]:
get_stop_ids_for_each_route_ids_number_of_shapes_per_direction_greater_than_1 = pd.merge(route_ids_number_of_shapes_per_direction_greater_than_1, simplified_trips_join_stop_times, on=['route_id','direction_id'], how='inner')
group_stop_ids_for_each_route_ids_number_of_shapes_per_direction_greater_than_1 = get_stop_ids_for_each_route_ids_number_of_shapes_per_direction_greater_than_1.groupby(['route_id','direction_id','shape_id'])['stop_id'].unique().reset_index(name='stop_ids')

,route_id,direction_id,shape_id,stop_ids
0,10-13168,0,100750_JUNE23,"[3202, 3217, 3232, 3227, 3231, 4716, 3212, 322..."
1,10-13168,0,100753_JUNE23,"[3203, 3202, 3217, 3232, 3227, 3231, 4716, 321..."
2,10-13168,0,100767_JUNE23,"[17013, 3222, 3226, 3210, 3220, 3225, 3215, 32..."
3,10-13168,0,100769_JUNE23,"[3203, 3202, 3217, 3232, 3227, 3231, 4716, 321..."
4,10-13168,0,100771_JUNE23,"[3202, 3217, 3232, 3227, 3231, 4716, 3212, 322..."
...,...,...,...,...
599,92-13168,1,920304_JUNE23,"[2228, 2256, 2254, 10763, 2223, 2248, 2257, 22..."
600,94-13168,0,940258_JUNE23,"[6535, 5128, 20139, 20140, 7238, 142450, 13602..."
601,94-13168,0,940259_JUNE23,"[6535, 5128, 20139, 20140, 7238, 142450, 13602..."
602,DSE-US,1,DSE-US-US-CF,"[63500001, 142043, 2155]"


### 3.2 Get the `stop_ids` that are not in the `route_id` and `stop_id` combination

We then get the `stop_id`s that are not in the `route_id` and `stop_id` combination. This will give us a dataframe called `stop_ids_not_in_route_id_and_stop_id_combination` with the following columns:
- `route_id`
- `direction_id`
- `stop_ids`



In [107]:
route_id_with_stop_id_differences = group_stop_ids_for_each_route_ids_number_of_shapes_per_direction_greater_than_1.groupby(['route_id','direction_id'])['stop_ids'].apply(lambda x: x.iloc[0] if len(x) == 1 else list(set(x.iloc[0]) - set(x.iloc[1]))).reset_index(name='stop_ids')
route_id_with_stop_id_differences

,route_id,direction_id,stop_ids
0,10-13168,0,[]
1,10-13168,1,[3203]
2,102-13168,0,[]
3,102-13168,1,"[13570, 1417, 10378, 10379, 10380, 10381, 1127..."
4,105-13168,0,"[16532, 2464, 70500001, 2465, 4518, 14378, 452..."
...,...,...,...
155,910-13168,1,"[5411, 65300039, 142216, 13802, 2603, 13803, 1..."
156,92-13168,0,"[3328, 3585, 3586, 3584, 1544, 10761, 10766, 1..."
157,92-13168,1,[]
158,94-13168,0,[]


### 3.3 Merge the `stop_ids` that are not in the `route_id` and `stop_id` combination with the joined data

In [108]:
combined_route_id_with_stop_differences_to_joined_data = pd.merge(group_route_id_by_distinct_direction_id_shape_ids, route_id_with_stop_id_differences, on=['route_id','direction_id'], how='inner')
combined_route_id_with_stop_differences_to_joined_data

,route_id,direction_id,shape_ids,stop_ids
0,10-13168,0,"[100750_JUNE23, 100753_JUNE23, 100767_JUNE23, ...",[]
1,10-13168,1,"[100751_JUNE23, 100756_JUNE23, 100794_JUNE23, ...",[3203]
2,102-13168,0,"[1020037_JUNE23, 1020038_JUNE23, 1020067_JUNE2...",[]
3,102-13168,1,"[1020072_JUNE23, 1020074_JUNE23, 1020075_JUNE23]","[13570, 1417, 10378, 10379, 10380, 10381, 1127..."
4,105-13168,0,"[1050266_JUNE23, 1050267_JUNE23, 1050268_JUNE2...","[16532, 2464, 70500001, 2465, 4518, 14378, 452..."
...,...,...,...,...
155,910-13168,1,"[9100209_JUNE23, 9100210_JUNE23]","[5411, 65300039, 142216, 13802, 2603, 13803, 1..."
156,92-13168,0,"[920224_JUNE23, 920274_JUNE23, 920275_JUNE23, ...","[3328, 3585, 3586, 3584, 1544, 10761, 10766, 1..."
157,92-13168,1,"[920225_JUNE23, 920299_JUNE23, 920300_JUNE23, ...",[]
158,94-13168,0,"[940258_JUNE23, 940259_JUNE23]",[]


### 3.4 Output the data to a csv file

We then output the data to a csv file called `route_analysis.csv`.

In [123]:
combined_route_id_with_stop_differences_to_joined_data.to_csv('output/route_analysis.csv', index=False)

### 4.1 Route Analysis by `shape_id`

We then output the data to a csv file called `route_analysis_by_shape_id.csv` that is grouped by `shape_id`.

In [113]:
get_list_differing_stop_ids_per_route_id_with_distinct_shape_ids = simplified_trips_join_stop_times.groupby(['route_id','direction_id','shape_id'])['stop_id'].unique().reset_index(name='stop_ids')
get_list_differing_stop_ids_per_route_id_with_distinct_shape_ids['number_of_stops_per_shape'] = get_list_differing_stop_ids_per_route_id_with_distinct_shape_ids['stop_ids'].apply(lambda x: len(x))
routes_and_stop_ids = get_list_differing_stop_ids_per_route_id_with_distinct_shape_ids
routes_and_stop_ids

,route_id,direction_id,shape_id,stop_ids,number_of_stops_per_shape
0,10-13168,0,100750_JUNE23,"[3202, 3217, 3232, 3227, 3231, 4716, 3212, 322...",89
1,10-13168,0,100753_JUNE23,"[3203, 3202, 3217, 3232, 3227, 3231, 4716, 321...",90
2,10-13168,0,100767_JUNE23,"[17013, 3222, 3226, 3210, 3220, 3225, 3215, 32...",56
3,10-13168,0,100769_JUNE23,"[3203, 3202, 3217, 3232, 3227, 3231, 4716, 321...",48
4,10-13168,0,100771_JUNE23,"[3202, 3217, 3232, 3227, 3231, 4716, 3212, 322...",47
...,...,...,...,...,...
671,DSE-HG,0,DSE-HG-DS,"[30005, 2321, 10855, 2320, 2322, 63500003]",6
672,DSE-HG,1,DSE-HG-HG,"[63500003, 10994, 10853, 2324, 10846, 30005]",6
673,DSE-US,0,DSE-US-DS,"[2155, 9221, 63500001, 63500004]",4
674,DSE-US,1,DSE-US-US-CF,"[63500001, 142043, 2155]",3


### 4.2 Output the data to a csv file

We then output the data to a csv file called `route_analysis_by_route_id.csv` that has the `shape_id` as rows.

In [124]:
routes_and_stop_ids.to_csv('output/route_analysis_per_shape_id.csv', index=False, sep=',')